<img src="https://github.com/IKNL/guidelines/blob/master/resources/logos/iknl_nl.png?raw=true" width=200 align="right">

# `validation_centralized_federated`
In this notebook, we validate our proposed federated GLM by comparing its performance against that of `R`'s `glm()` (i.e., centralized).

The families that are shown in this notebook are: 

* `gaussian(link = "identity")`: Linear regression
* `poisson(link = "log")`: Poisson regression
* `binomial(link = "logit")`: Logistic regression


## Requirements
In order to be able to run it properly, make sure that the following libraries are installed:

* `vtg` - Basic `vantage6` tools. These will allow you to use the mock client. They can be installed with the command:

   `devtools::install_github('IKNL/vtg')`
   
   
   
* `vtg.glm` - The actual federated GLM algorithm. It can be installed with the command:

   `devtools::install_github(repo='IKNL/vantage6-algorithms', ref='glm', subdir='models/glm/src')`

## Preliminaries

In [1]:
# Imports
library('stargazer')


Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 




## Data
The `.csv` files for the data can be generated running the accompanying (Python) script `create_data.py`. Each family (i.e., model type) has its corresponding data. In all cases, they represent a horizontally-partitioned scenario (i.e., all parties have the same features from different individuals).

## Validation

#### The `validate()` function
This function compares the centralized and federated results. The most important values (coefficients, standard errors, $p$- and $z$-values are printed to the console using `stargazer`. Additionally, it checks if other values (such as conversion, formula, family, etc.) are equal between them (shown as `TRUE`) as a sanity check.

In [2]:
validate <- function(results_centralized, results_federated_plain){
  
  results_centralized_summary <- summary.glm(results_centralized)
  results_federated <- vtg.glm::as.GLM(results_federated_plain)
  
  precision <- 10
  options(digits=precision)
  my_round <- function(x) {round(x, digits = 10)}
  
  validation_result <- list()
    
  # Coefficients
  coeffs_centralized = sapply(results_centralized$coefficients, my_round, USE.NAMES = TRUE)    
  coeffs_federated = sapply(results_federated_plain$coefficients, my_round, USE.NAMES = TRUE)
  
  df = data.frame(coeff_c = c(coeffs_centralized),
                  coeff_f = c(coeffs_federated))  
    
  # Standard error
  stderr_centralized = sapply(results_centralized_summary$coefficients[, 'Std. Error'], my_round, USE.NAMES = TRUE)
  stderr_federated = sapply(results_federated_plain$Std.Error, my_round, USE.NAMES = TRUE)
    
  df['stderr_c'] <- c(stderr_centralized)
  df['stderr_f'] <- c(stderr_federated)
  

  if(results_centralized_summary$family$family %in% c('poisson', 'binomial')) {letter='z'} 
  else if(results_centralized_summary$family$family=='gaussian') {letter='t'}
  
  # p-value
  pval_centralized = sapply(results_centralized_summary$coefficients[, sprintf("Pr(>|%s|)", letter)], my_round, USE.NAMES = TRUE)
  pval_federated = sapply(results_federated_plain$pvalue, my_round, USE.NAMES = TRUE)
   
  df['pvalues_c'] <- c(pval_centralized)
  df['pvalues_f'] <- c(pval_federated)
    
    
  # z-value
  zval_centralized = sapply(results_centralized_summary$coefficients[, sprintf("%s value", letter)], my_round, USE.NAMES = TRUE)
  zval_federated = sapply(results_federated_plain$zvalue, my_round, USE.NAMES = TRUE)
  
  df['zvalues_c'] <- c(zval_centralized)
  df['zvalues_f'] <- c(zval_federated)
    
  print("LaTeX")
  stargazer(df, type="latex", summary=FALSE)
  print("Text")
  stargazer(df, type="text", summary=FALSE)
    
    
  # Converged
  validation_result['converged'] = results_federated_plain$converged==results_centralized$converged
  print(validation_result['converged'])
    
  # Dispersion
  FL_dispersion = sapply(results_federated_plain$dispersion, my_round, USE.NAMES = TRUE)
  central_dispersion = sapply(results_centralized_summary$dispersion, my_round, USE.NAMES = TRUE)
  validation_result['dispersion'] = FL_dispersion==central_dispersion
  
  # Formula
  validation_result['formula'] = results_federated_plain$formula==results_centralized$formula
  
  # Family
  FL_family = jsonlite::toJSON(results_federated_plain$family, auto_unbox = TRUE, force = T)
  central_family = jsonlite::toJSON(results_centralized$family, auto_unbox = TRUE, force = T)
  validation_result['family'] = FL_family==central_family
  
  # null.deviance
  FL_nulldeviance = sapply(results_federated_plain$null.deviance, my_round, USE.NAMES = TRUE)
  central_nulldeviance = sapply(results_centralized$null.deviance, my_round, USE.NAMES = TRUE)
  validation_result['null.deviance'] = FL_nulldeviance==central_nulldeviance
  
  # DoF
  FL_dof = sapply(results_federated$df.residual, my_round, USE.NAMES = TRUE)
  central_dof = sapply(results_centralized$df.residual, my_round, USE.NAMES = TRUE)
  validation_result['df.residual'] = FL_dof==central_dof
  
  # null.dof
  FL_null = sapply(results_federated$df.null, my_round, USE.NAMES = TRUE)
  central_null = sapply(results_centralized$df.null, my_round, USE.NAMES = TRUE)
  validation_result['df.null'] = FL_null==central_null
    
  validation_result
}

### Linear
#### Centralized analysis

In [3]:
# Housekeeping
if(exists('results_centralized')){
    rm(results_centralized)
}
if(exists('results_federated_plain')){
    rm(results_federated_plain)
}

datasets <- list(
  read.csv('../data/linear_party1.csv'),
  read.csv('../data/linear_party2.csv'),
  read.csv('../data/linear_party3.csv')
)
datasets_combined <- do.call(rbind.data.frame, datasets)

results_centralized <- glm(data=datasets_combined, formula = y ~ x1 + x2, family=gaussian(link = 'identity'))

#### Federated analysis

In [4]:
client <- vtg::MockClient$new(datasets, 'vtg.glm')
results_federated_plain <- vtg.glm::dglm(client, formula = y ~ x1 + x2, family=gaussian(link = 'identity'), tol=1e-08, maxit=25)

-----------------------------------------
  Welcome to the vantage6 Infrastructure
-----------------------------------------




INFO  [18:25:41.984]  
INFO  [18:25:42.013] ############################################### 
INFO  [18:25:42.021] # Starting iteration 0 
INFO  [18:25:42.024] ############################################### 
INFO  [18:25:42.026]  
INFO  [18:25:42.029] 0.1 - RPC Node Beta 
DEBUG [18:25:42.032] ** Mocking call to "node_beta" ** 
DEBUG [18:25:42.035] Regular call 
DEBUG [18:25:42.039] there are 3 datasets ..  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [18:25:42.107] Log for site 1: 
DEBUG [18:25:42.109] Running a (mere) regular container. 
Calling RPC_node_beta 
DEBUG [18:25:42.046] Initializing node beta... 
DEBUG [18:25:42.056] First iteration. Initializing variables. 
DEBUG [18:25:42.059] Calculating the Betas.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


...

DEBUG [18:25:42.422] Log for site 1: 
DEBUG [18:25:42.424] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [18:25:42.413] Starting node deviance. 
DEBUG [18:25:42.419] Updating the variables for node deviance.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [18:25:42.437] Log for site 2: 
DEBUG [18:25:42.440] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [18:25:42.427] Starting node deviance. 
DEBUG [18:25:42.435] Updating the variables for node deviance.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [18:25:42.452] Log for site 3: 
DEBUG [18:25:42.455] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [18:25:42.443] Starting node deviance. 
DEBUG [18:25:42.450] Updating the variables for node deviance.  
DEBUG [18:25:42.458]   - [DONE] 
INFO  [18:25:42.461] 1.4 - Master deviance 
DEBUG [18:25:42.463] Starting master deviance. 
DEBUG [18:25:42.466]   - [DONE] 
INFO  [18:25:42.468] 1.5 - Termination conditions 
DEBUG [18:25:42.470]   - [CONVERGED] 
DEBUG [18:25:42.474] Preparing output 


#### Comparison

In [5]:
val_linear_res = validate(results_centralized, results_federated_plain)
print(val_linear_res)

[1] "LaTeX"

% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Mon, May 09, 2022 - 18:25:42
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}} ccccccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & coeff\_c & coeff\_f & stderr\_c & stderr\_f & pvalues\_c & pvalues\_f & zvalues\_c & zvalues\_f \\ 
\hline \\[-1.8ex] 
(Intercept) & $0.029$ & $0.029$ & $0.045$ & $0.045$ & $0.512$ & $0.512$ & $0.656$ & $0.656$ \\ 
x1 & $0.231$ & $0.231$ & $0.018$ & $0.018$ & $0$ & $0$ & $12.694$ & $12.694$ \\ 
x2 & $0.494$ & $0.494$ & $0.019$ & $0.019$ & $0$ & $0$ & $26.633$ & $26.633$ \\ 
\hline \\[-1.8ex] 
\end{tabular} 
\end{table} 
[1] "Text"

            coeff_c coeff_f stderr_c stderr_f pvalues_c pvalues_f zvalues_c zvalues_f
-------------------------------------------------------------------------------------
(Intercept)  0.029   0.029   0.045    0.045     0.512     0.512     0.65

### Poisson
#### Centralized

In [6]:
# Housekeeping
if(exists('results_centralized')){
    rm(results_centralized)
}
if(exists('results_federated_plain')){
    rm(results_federated_plain)
}

datasets <- list(
  read.csv('../data/poisson_party1.csv'),
  read.csv('../data/poisson_party2.csv'),
  read.csv('../data/poisson_party3.csv')
)
datasets_combined <- do.call(rbind.data.frame, datasets)

results_centralized <- glm(data=datasets_combined, formula = y ~ x1 + x2, family=poisson(link = "log"))

#### Federated

In [7]:
client <- vtg::MockClient$new(datasets, "vtg.glm")
results_federated_plain <- vtg.glm::dglm(client, formula = y ~ x1 + x2, family=poisson(link = "log"), tol=1e-08, maxit=25)

DEBUG [18:25:42.739] Initializing... 
INFO  [18:25:42.742]  
INFO  [18:25:42.744] ############################################### 
INFO  [18:25:42.747] # Starting iteration 0 
INFO  [18:25:42.749] ############################################### 
INFO  [18:25:42.752]  
INFO  [18:25:42.754] 0.1 - RPC Node Beta 
DEBUG [18:25:42.755] ** Mocking call to "node_beta" ** 
DEBUG [18:25:42.758] Regular call 
DEBUG [18:25:42.764] there are 3 datasets ..  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [18:25:42.779] Log for site 1: 
DEBUG [18:25:42.782] Running a (mere) regular container. 
Calling RPC_node_beta 
DEBUG [18:25:42.766] Initializing node beta... 
DEBUG [18:25:42.773] First iteration. Initializing variables. 
DEBUG [18:25:42.776] Calculating the Betas.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [18:25:42.796] Log for site 2: 
DEBUG [18:25:42.799] Running a (mere) regular container. 
Calling RPC_node_beta 
DEBUG [18:25:42.784] Initializing node beta... 
DEBUG [18:25:42.790] First iteration. Initializing variables. 
DEBUG [18:25:42.793] Calculating the Betas.  


...

DEBUG [18:25:43.512] Log for site 1: 
DEBUG [18:25:43.515] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [18:25:43.501] Starting node deviance. 
DEBUG [18:25:43.509] Updating the variables for node deviance.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [18:25:43.530] Log for site 2: 
DEBUG [18:25:43.533] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [18:25:43.519] Starting node deviance. 
DEBUG [18:25:43.526] Updating the variables for node deviance.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [18:25:43.547] Log for site 3: 
DEBUG [18:25:43.550] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [18:25:43.536] Starting node deviance. 
DEBUG [18:25:43.543] Updating the variables for node deviance.  
DEBUG [18:25:43.553]   - [DONE] 
INFO  [18:25:43.557] 4.4 - Master deviance 
DEBUG [18:25:43.559] Starting master deviance. 
DEBUG [18:25:43.563]   - [DONE] 
INFO  [18:25:43.565] 4.5 - Termination conditions 
DEBUG [18:25:43.568]   - [CONVERGED] 
DEBUG [18:25:43.571] Preparing output 


#### Comparison

In [8]:
val_poisson_res = validate(results_centralized, results_federated_plain)
print(val_poisson_res)

[1] "LaTeX"

% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Mon, May 09, 2022 - 18:25:43
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}} ccccccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & coeff\_c & coeff\_f & stderr\_c & stderr\_f & pvalues\_c & pvalues\_f & zvalues\_c & zvalues\_f \\ 
\hline \\[-1.8ex] 
(Intercept) & $0.470$ & $0.470$ & $0.021$ & $0.021$ & $0$ & $0$ & $22.124$ & $22.124$ \\ 
x1 & $0.214$ & $0.214$ & $0.007$ & $0.007$ & $0$ & $0$ & $29.585$ & $29.585$ \\ 
x2 & $0.524$ & $0.524$ & $0.007$ & $0.007$ & $0$ & $0$ & $74.560$ & $74.560$ \\ 
\hline \\[-1.8ex] 
\end{tabular} 
\end{table} 
[1] "Text"

            coeff_c coeff_f stderr_c stderr_f pvalues_c pvalues_f zvalues_c zvalues_f
-------------------------------------------------------------------------------------
(Intercept)  0.470   0.470   0.021    0.021       0         0      22.124    2

### Logistic

#### Centralized

In [9]:
# Housekeeping
if(exists('results_centralized')){
    rm(results_centralized)
}
if(exists('results_federated_plain')){
    rm(results_federated_plain)
}

datasets <- list(
  read.csv('../data/logistic_party1.csv'),
  read.csv('../data/logistic_party2.csv'),
  read.csv('../data/logistic_party3.csv')
)
datasets_combined <- do.call(rbind.data.frame, datasets)
results_centralized <- glm(data=datasets_combined, formula = y ~ x1 + x2, family=binomial(link = "logit"))

#### Federated

In [10]:
client <- vtg::MockClient$new(datasets, "vtg.glm")
results_federated_plain <- vtg.glm::dglm(client, formula = y ~ x1 + x2, family=binomial(link = "logit"), tol=1e-08, maxit=25)

DEBUG [18:25:43.790] Initializing... 
INFO  [18:25:43.794]  
INFO  [18:25:43.799] ############################################### 
INFO  [18:25:43.802] # Starting iteration 0 
INFO  [18:25:43.805] ############################################### 
INFO  [18:25:43.808]  
INFO  [18:25:43.812] 0.1 - RPC Node Beta 
DEBUG [18:25:43.815] ** Mocking call to "node_beta" ** 
DEBUG [18:25:43.822] Regular call 
DEBUG [18:25:43.825] there are 3 datasets ..  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [18:25:43.845] Log for site 1: 
DEBUG [18:25:43.848] Running a (mere) regular container. 
Calling RPC_node_beta 
DEBUG [18:25:43.828] Initializing node beta... 
DEBUG [18:25:43.836] First iteration. Initializing variables. 
DEBUG [18:25:43.841] Calculating the Betas.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [18:25:43.874] Log for site 2: 
DEBUG [18:25:43.878] Running a (mere) regular container. 
Calling RPC_node_beta 
DEBUG [18:25:43.851] Initializing node beta... 
DEBUG [18:25:43.862] First iteration. Initializing variables. 
DEBUG [18:25:43.870] Calculating the Betas.  


...

DEBUG [18:25:44.936] Log for site 1: 
DEBUG [18:25:44.939] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [18:25:44.923] Starting node deviance. 
DEBUG [18:25:44.930] Updating the variables for node deviance.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [18:25:44.953] Log for site 2: 
DEBUG [18:25:44.956] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [18:25:44.943] Starting node deviance. 
DEBUG [18:25:44.950] Updating the variables for node deviance.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [18:25:44.970] Log for site 3: 
DEBUG [18:25:44.972] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [18:25:44.959] Starting node deviance. 
DEBUG [18:25:44.967] Updating the variables for node deviance.  
DEBUG [18:25:44.976]   - [DONE] 
INFO  [18:25:44.979] 5.4 - Master deviance 
DEBUG [18:25:44.981] Starting master deviance. 
DEBUG [18:25:44.984]   - [DONE] 
INFO  [18:25:44.986] 5.5 - Termination conditions 
DEBUG [18:25:44.989]   - [CONVERGED] 
DEBUG [18:25:44.991] Preparing output 


#### Comparison

In [11]:
val_binomial_res = validate(results_centralized, results_federated_plain)
print(val_binomial_res)

[1] "LaTeX"

% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Mon, May 09, 2022 - 18:25:45
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}} ccccccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & coeff\_c & coeff\_f & stderr\_c & stderr\_f & pvalues\_c & pvalues\_f & zvalues\_c & zvalues\_f \\ 
\hline \\[-1.8ex] 
(Intercept) & $0.388$ & $0.388$ & $0.065$ & $0.065$ & $0$ & $0$ & $5.986$ & $5.986$ \\ 
x1 & $2.511$ & $2.511$ & $0.093$ & $0.093$ & $0$ & $0$ & $27.133$ & $27.133$ \\ 
x2 & $$-$0.438$ & $$-$0.438$ & $0.049$ & $0.049$ & $0$ & $0$ & $$-$8.920$ & $$-$8.920$ \\ 
\hline \\[-1.8ex] 
\end{tabular} 
\end{table} 
[1] "Text"

            coeff_c coeff_f stderr_c stderr_f pvalues_c pvalues_f zvalues_c zvalues_f
-------------------------------------------------------------------------------------
(Intercept)  0.388   0.388   0.065    0.065       0         0       5.

## References
For more information, please refer to our paper:

> M. Cellamare, A.J. van Gestel, H. Alradhi, F. Martin, A. Moncada-Torres. "A Federated Generalized Linear Model for Privacy-Preserving Analysis". Under review